In [117]:
import json
from guess_language import guess_language
import re
import docx

In [118]:
doc = docx.Document("stern-translation.docx")

In [119]:
all_paras = doc.paragraphs
sections = []
remarkNums = []

In [120]:
for para in all_paras:
    runs = para.runs
    for run in runs:
        if run.italic:
            text = run.text
            run.text = '<em>'+text+'</em>'
        if run.underline:
            print(run.text)
            text = run.text
            underline_type = str(run.underline)
            if underline_type == 'True':
                run.text = "<span class='underline_single'>"+text+"</span>"
            if underline_type == 'DOUBLE (3)':
                run.text = "<span class='underline_double'>"+text+"</span>"
    if (re.search("\S", para.text)):
        sections.append(para.text)

In [121]:
bigList = []
node = []
for section in sections:
    if (re.search("^[0-9]\.[0-9]+", section)):
        if node:
            bigList.append(node)
            node = []
    node.append(section)

In [122]:
def remove_ger(List):
    eng = ''
    tlpNumber = ''
    remark = {}
    for item in List:
        if (guess_language(item) == 'en' or guess_language(item) == 'el'):
            eng += '<p>'+item+'</p>'
        if (re.search("^[0-9]\.[0-9]+", item)):
            tlpNumber = item
    remark['eng'] = eng
    remark['tlp-number'] = tlpNumber.strip()
    return remark

In [123]:
sternJson = []
for el in bigList:
    if (re.search("^[0-9]\.[0-9]+", el[0])):
        sternJson.append(remove_ger(el))

In [124]:
with open('stern-translation.json', 'w', encoding='utf8') as f:
    json.dump(sternJson, f, indent=4, ensure_ascii=False)

# merge files

In [148]:
pt = open('ptSections.json')
recto = open('recto.json')
data = json.load(pt)
ptSections = data['sections']
rectoSections = json.load(recto)

In [149]:
def filterStern(List, key, val):
    for l in List:
        if (l[key] == val):
            return l

In [169]:
def normalizeTlpNum(tlpNum):
    num = tlpNum.strip()
    match = re.match("[^\s]+", num)
    try:
        num3 = match.group(0).replace("·", ".")
        num4 = num3.replace("*","")
        num5 = num4.replace("+","")
    except:
        num5 = ""    
    return num5

In [182]:
def findPtfromTlpNum(tlpNum):
    tlp = str(tlpNum)
    for section in ptSections:
        tlpRef = section['tlp'][0]
        tlpRef2 = str(tlpRef)
        if (tlp == tlpRef2):
            return section['str']

In [170]:
print(filterStern(sternJson, 'tlp-number', '2.012'))

{'eng': '<p>In logic nothing is coincidental: if a thing <em>can</em> occur in a state of affairs, the possibility of the state of affairs must already be foreshadowed in the thing.</p>', 'tlp-number': '2.012'}


In [171]:
newList = []

In [193]:
for section in rectoSections:
    tlpNum = normalizeTlpNum(section['tlp-number'])
    sternTranslation = filterStern(sternJson, 'tlp-number', tlpNum)
    sternPt = findPtfromTlpNum(tlpNum)
    new = section
    new['stern'] = ""
    new['pt-stern'] = ""
    if (sternTranslation):
        new['stern'] = sternTranslation['eng']
    if (sternPt):
        new['pt-stern'] = '<p>'+sternPt+'</p>'
    newList.append(new)

In [194]:
with open('recto-experimental.json', 'w', encoding='utf8') as f:
    json.dump(newList, f, indent=4, ensure_ascii=False)